<a href="https://colab.research.google.com/github/NathanDietrich/Artificial-Intelligence-and-Machine-Learning-portfolio/blob/main/HyperparameterTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
# Load the CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

In [5]:
print(f"Training images shape: {train_images.shape}")
print(f"Training labels shape: {train_labels.shape}")
print(f"Test images shape: {test_images.shape}")
print(f"Test labels shape: {test_labels.shape}")

Training images shape: (50000, 32, 32, 3)
Training labels shape: (50000, 1)
Test images shape: (10000, 32, 32, 3)
Test labels shape: (10000, 1)


In [6]:
# Normalize the pixel values to [0,1]
train_images = train_images.astype('float32')/255
test_images = test_images.astype('float32') / 255.0

In [7]:
#one hot encoding
num_classes = 10
train_labels_encoded = to_categorical(train_labels, num_classes)
test_labels_encoded = to_categorical(test_labels, num_classes)

In [8]:
print(f"Encoded training labels shape: {train_labels_encoded.shape}")
print(f"Encoded test labels shape: {test_labels_encoded.shape}")

Encoded training labels shape: (50000, 10)
Encoded test labels shape: (10000, 10)


In [9]:
X_train, X_val, y_train, y_val = train_test_split(
    train_images,
    train_labels_encoded,
    test_size=0.2,
    random_state=42,
    stratify=train_labels
)

In [ ]:
def resize_images(images, size=(224, 224)):
    return np.array([tf.image.resize(image, size).numpy() for image in images])

X_train_resized = resize_images(X_train)
X_val_resized = resize_images(X_val)
test_images_resized = resize_images(test_images)

In [ ]:
# Load the pre-trained VGG16 model (excluding the top classifier)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.summary()

In [ ]:
# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
x = Dense(256, activation='relu')(x)

In [ ]:
# Add the final classification layer with 10 units (for CIFAR-10 classes) and softmax activation
predictions = Dense(10, activation='softmax')(x)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val)
)